In [1]:
import os
import numpy as np
import pandas as pd

from keras.layers import *
from keras import backend as K
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.models import Model,load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau,EarlyStopping

from scipy.misc import toimage,imresize
from skimage import exposure
from PIL import Image
import cv2

Using TensorFlow backend.


### Image generators

In [2]:
spirals_train_folder = '../input/drawings/spiral/training'
spirals_val_folder = '../input/drawings/spiral/testing'
waves_train_folder = '../input/drawings/wave/training'
waves_val_folder = '../input/drawings/wave/testing'

batch_size = 24

# histogram equalizer
def eqz_plz(img):
    return exposure.equalize_hist(img)


spiral_datagen = ImageDataGenerator(rotation_range=360, # they're spirals.
                                    width_shift_range=0.1,
                                    height_shift_range=0.1,
                                    brightness_range=(0.5,1.5),
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    preprocessing_function=eqz_plz,
                                    vertical_flip=True)

wave_datagen = ImageDataGenerator(rotation_range=5,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  brightness_range=(0.5,1.5),
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  preprocessing_function=eqz_plz,
                                  vertical_flip=True)


spiral_train_generator = spiral_datagen.flow_from_directory(directory=os.path.abspath(spirals_train_folder),
                                                            target_size=(256, 256),
                                                            color_mode="grayscale",
                                                            batch_size=batch_size,
                                                            class_mode="binary",
                                                            shuffle=True,
                                                            seed=666)

spiral_val_generator = spiral_datagen.flow_from_directory(directory=os.path.abspath(spirals_val_folder),
                                                            target_size=(256, 256),
                                                            color_mode="grayscale",
                                                            batch_size=batch_size,
                                                            class_mode="binary",
                                                            shuffle=True,
                                                            seed=710)

wave_train_generator = wave_datagen.flow_from_directory(directory=os.path.abspath(waves_train_folder),
                                                        target_size=(256, 512), # HxW in machine learning, WxH in computer vision
                                                        color_mode="grayscale",
                                                        batch_size=batch_size,
                                                        class_mode="binary",
                                                        shuffle=True,
                                                        seed=420)

wave_val_generator = wave_datagen.flow_from_directory(directory=os.path.abspath(waves_val_folder),
                                                        target_size=(256, 512),
                                                        color_mode="grayscale",
                                                        batch_size=batch_size,
                                                        class_mode="binary",
                                                        shuffle=True,
                                                        seed=420)

Found 72 images belonging to 2 classes.
Found 30 images belonging to 2 classes.
Found 72 images belonging to 2 classes.
Found 30 images belonging to 2 classes.


In [3]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss',patience=12,min_lr=1e-9,verbose=1)
early_stop = EarlyStopping(monitor='val_loss',patience=16,verbose=1)

## DNN

In [4]:
K.clear_session()

def nopamine_model(mode):
    if (mode == 'spirals') or (mode == 'spiral'):
        input_layer = Input(shape=(256,256,1),name=f'{mode}_input_layer')
    elif (mode == 'waves') or (mode == 'wave'):
        input_layer = Input(shape=(256,512,1),name=f'{mode}_input_layer')

    m1 = Conv2D(256,(5,5),dilation_rate=4,kernel_initializer='glorot_normal',kernel_regularizer=l2(0.001),activation='relu',padding='same')(input_layer)
    p1 = MaxPool2D((9,9),strides=3)(m1)
    m2 = Conv2D(128,(5,5),dilation_rate=2,kernel_initializer='glorot_normal',kernel_regularizer=l2(0.001),activation='relu',padding='same')(p1)
    p2 = MaxPool2D((7,7),strides=3)(m2)
    m3 = Conv2D(64,(3,3),kernel_initializer='glorot_normal',kernel_regularizer=l2(0.001),activation='relu',padding='same')(p2)
    p3 = MaxPool2D((5,5),strides=2)(m3)
    g1=Dropout(0.2)(p3)
    f1 = Flatten()(g1)
    d1 = Dense(666,activation='relu')(f1)
    g2=Dropout(0.5)(d1)
    d2 = Dense(1,activation='sigmoid')(g2)
    

    
    this_model = Model(input_layer,d2)
    this_model.summary()
    return this_model

### Network for spirals

In [5]:
spiral_model = nopamine_model(mode='spirals') # early stopping epoch 89: val_loss 0.4796, val_acc 0.8274
spiral_model.compile(optimizer=Adam(lr=3.15e-5), loss='binary_crossentropy', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
spirals_input_layer (InputLa (None, 256, 256, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 256, 256, 256)     6656      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 83, 83, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 83, 83, 128)       819328    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 26, 26, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)    

In [6]:
result=spiral_model.fit_generator(spiral_train_generator,
                           validation_data=spiral_val_generator,
                           epochs=666,
                           steps_per_epoch=(2000//batch_size),
                           validation_steps=(800//batch_size),
                           callbacks=[reduce_lr,early_stop],
                           verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/666


/opt/conda/lib/python3.6/site-packages/skimage/exposure/exposure.py:124: UserWarning: This might be a color image. The histogram will be computed on the flattened image. You can instead apply this function to each color channel.
  warn("This might be a color image. The histogram will be "


83/83 [==============================] - 37s 448ms/step - loss: 0.9469 - acc: 0.4885 - val_loss: 0.9340 - val_acc: 0.5317
Epoch 2/666
83/83 [==============================] - 29s 350ms/step - loss: 0.9287 - acc: 0.4920 - val_loss: 0.9183 - val_acc: 0.5041
Epoch 3/666
83/83 [==============================] - 30s 356ms/step - loss: 0.9132 - acc: 0.4885 - val_loss: 0.9042 - val_acc: 0.5099
Epoch 4/666
83/83 [==============================] - 29s 348ms/step - loss: 0.8977 - acc: 0.5166 - val_loss: 0.8908 - val_acc: 0.5082
Epoch 5/666
83/83 [==============================] - 29s 355ms/step - loss: 0.8868 - acc: 0.4970 - val_loss: 0.8793 - val_acc: 0.5000
Epoch 6/666
83/83 [==============================] - 29s 345ms/step - loss: 0.8740 - acc: 0.5221 - val_loss: 0.8684 - val_acc: 0.5391
Epoch 7/666
83/83 [==============================] - 29s 354ms/step - loss: 0.8646 - acc: 0.5100 - val_loss: 0.8572 - val_acc: 0.6012
Epoch 8/666
83/83 [==============================] - 29s 347ms/step - loss

Epoch 62/666
83/83 [==============================] - 29s 355ms/step - loss: 0.4133 - acc: 0.8509 - val_loss: 0.4735 - val_acc: 0.7963
Epoch 63/666
83/83 [==============================] - 29s 348ms/step - loss: 0.4154 - acc: 0.8499 - val_loss: 0.4763 - val_acc: 0.8155
Epoch 64/666
83/83 [==============================] - 29s 349ms/step - loss: 0.4097 - acc: 0.8544 - val_loss: 0.5161 - val_acc: 0.7860
Epoch 65/666
83/83 [==============================] - 29s 354ms/step - loss: 0.4141 - acc: 0.8534 - val_loss: 0.4851 - val_acc: 0.7956
Epoch 66/666
83/83 [==============================] - 29s 351ms/step - loss: 0.4045 - acc: 0.8534 - val_loss: 0.4688 - val_acc: 0.8354
Epoch 67/666
83/83 [==============================] - 29s 350ms/step - loss: 0.3986 - acc: 0.8584 - val_loss: 0.5021 - val_acc: 0.7956
Epoch 68/666
83/83 [==============================] - 29s 347ms/step - loss: 0.4016 - acc: 0.8544 - val_loss: 0.4270 - val_acc: 0.8313
Epoch 69/666
83/83 [==============================] - 3

In [20]:
# here's how to load/save
spiral_model.save('/kaggle/working/model_spirals.h5')